# Data Collection and Exploration

Here we will load in the data from youtube, where collecting cc texts from relevant search queries is the main goal.

In [1]:
import os
from dotenv import load_dotenv
from youtube_transcript_api import YouTubeTranscriptApi
from googleapiclient.discovery import build

Change directories to get access to env file.

In [2]:
os.chdir("../")

In [3]:
pwd

'c:\\Users\\RaviB\\GitHub\\TechKnowBot'

Build YouTube connector.

In [4]:
load_dotenv()

youtube_api_key = os.getenv('GCP_YOUTUBE_API_KEY')

Take old code from SentiRec Analytics project: https://github.com/RavinderRai/SentiRec-Analytics/blob/main/modules/YouTubeReviewScraper.py.

In [5]:
class YouTubeReviewData:    
    def __init__(self, api_key):
        self.api_key = api_key
        self.youtube = build('youtube', 'v3', developerKey=api_key)
        
    def search_videos(self, search_query, max_results=5, individual_review=False):
        """
        Search for YouTube videos based on a given query and retrieve additional information including closed captions.

        Parameters:
        - search_query (str): The search query used to find relevant videos on YouTube.
        - max_results (int): The maximum number of videos to retrieve. Defaults to 5.
        - individual_review (bool): If the search query is for reviews of a specific product, then set this to True, 
        to ignore videos with VS in their titles as that indicates that the reviews isn't for just the individual product itself.

        Returns:
        List[dict]: A list of dictionaries, each containing information about a video, including:
            - 'video_id' (str): The unique identifier for the video.
            - 'title' (str): The title of the video.
            - 'video_link' (str): The YouTube link to the video.
            - 'channel_name' (str): The name of the channel that uploaded the video.
            - 'cc_text' (str): The closed captions text for the video. This is the review text.

        Note:
        - Videos with titles containing specific strings ('VS', 'vs', 'Vs') are excluded, as they indicate videos that aren't reviews specific to the  
        product in the search query.
        - The 'cc_text' field may contain an empty string if closed captions are not available.
        """       
        
        search_response = self.youtube.search().list(
            q=search_query,
            type='video',
            part='id, snippet',
            maxResults=max_results
        ).execute()        
        
        videos_info = []
        for result in search_response.get('items', []):
            video_id = result['id']['videoId']
            title = result['snippet']['title']
            video_link = f'https://www.youtube.com/watch?v={video_id}'
            channel_name = result['snippet']['channelTitle']

            # Check and remove unwanted titles
            strings_to_check = ["VS", "vs", "Vs"] if individual_review else []
            if not any(s in title for s in strings_to_check):
                review_text = self.fetch_captions(video_id)
                videos_info.append({
                    'video_id': video_id,
                    'title': title,
                    'video_link': video_link,
                    'channel_name': channel_name,
                    'cc_text': review_text
                })

        return videos_info
    
    def fetch_captions(self, video_id):
        """
        Get the closed captions. 

        Parameters:
        - video_id (str): The video id which is obtained in search_videos.
        
        Returns:
        String: Closed caption text of a youtube video
        """
        try:
            # Retrieve the transcript for the video
            transcript = YouTubeTranscriptApi.get_transcript(video_id)

            cc_text = ""

            # Concatenate the transcript text
            for entry in transcript:
                cc_text += ' ' + entry['text']
                
            cc_text = cc_text.replace('\n', ' ')
            return cc_text

        except Exception as e:
            print(f"An error occurred: {str(e)}")

In [6]:
youtube = YouTubeReviewData(youtube_api_key)

In [7]:
query_ex = "top ten laptops"
search_ex = youtube.search_videos(query_ex, 5)

In [8]:
print(search_ex[0]['cc_text'])

 this year we tested over 70 different laptops by far a record for this channel we tested small ones we tested big ones we tested cheap ones and we tested expensive ones a huge variety from all kinds of Manufacturers and when you use these laptops side by side just like we do it becomes so obvious which laptops are great and which are completely mediocre well today is the day that we countd down the top 10 laptops that we tested in 2023 if you are planning to buy one of these lap tops you'll obviously want to buy them at the best possible price so check the links below the video our team scours the internet to find the best deals and we update them daily plus if new laptops are released after this video that we end up liking even better we'll include them down there too number 10 the HB Pavilion plus 14 in for a price of around $729 you get a crazy amount for the money it will be configured with an OLED panel at that price which is bright vibrant color accurate has a fast refresh rate 

## Extracting Objects

After searching for videos with the best of for certain products, now we want to extract what those products are. We can do this with named entity recognition (NER).

In [9]:
import spacy
ner = spacy.load("en_core_web_sm")

In [10]:
search_ex[0]

{'video_id': 'X35QRgsHhF0',
 'title': 'The Best Laptops (early 2024)!',
 'video_link': 'https://www.youtube.com/watch?v=X35QRgsHhF0',
 'channel_name': 'Just Josh',
 'cc_text': " this year we tested over 70 different laptops by far a record for this channel we tested small ones we tested big ones we tested cheap ones and we tested expensive ones a huge variety from all kinds of Manufacturers and when you use these laptops side by side just like we do it becomes so obvious which laptops are great and which are completely mediocre well today is the day that we countd down the top 10 laptops that we tested in 2023 if you are planning to buy one of these lap tops you'll obviously want to buy them at the best possible price so check the links below the video our team scours the internet to find the best deals and we update them daily plus if new laptops are released after this video that we end up liking even better we'll include them down there too number 10 the HB Pavilion plus 14 in for a

In [11]:
cc_text_ex = search_ex[0]['cc_text']
doc = ner(cc_text_ex)

In [12]:
for ent in doc.ents:
    if ent.label_ in ["ORG", "PRODUCT"]:
        print(ent)

AMD N4
Intel
CPU
MacBook Air
AMD
AMD
Intel
AMD
Intel
Intel
AMD
Tinker
Apple MacBook
M2
MacBook
Apple
Roundup
Intel
4090 Graphics
Asus
MacBook Air
the Creature Comforts
Lenovo
Pro 9i
Pro 9i
Intel
Exquisite
Macbook Pros
Hardware
the Perfect Blend of power
Apple
M3
M3
M3
M2
M3 Max
Apple
M3 Max
the M3 Pro
AMD
AMD
Intel
Legion
Intel
Apple
M3 Pro
Apple


In [13]:
for i, ent in enumerate(doc.ents):
    if (ent.label_ == "ORG") and (doc.ents[i+1].label_ == "PRODUCT"):
        print(ent, doc.ents[i+1])

Apple MacBook M2
Lenovo Pro 9i
Apple M3 Max


Trying spacy-LLM integration for a better NER method, since the basic version above is missing a lot. Here is a guide on how to do this: https://github.com/explosion/spacy-llm/tree/main/usage_examples/ner_v3_openai.

In [14]:
nlp = spacy.blank("en")

In [15]:
from spacy_llm.util import assemble

In [16]:
nlp = assemble("notebooks/config.cfg")
doc = nlp(cc_text_ex)
print(doc.ents)

c:\Users\RaviB\anaconda3\envs\TechKnow\lib\site-packages\spacy_llm\models\rest\openai\model.py:25: UserWarning: Could not find the API key to access the OpenAI API. Ensure you have an API key set up via https://platform.openai.com/account/api-keys, then make it available as an environment variable 'OPENAI_API_KEY'.
  warnings.warn(
c:\Users\RaviB\anaconda3\envs\TechKnow\lib\site-packages\spacy_llm\models\rest\openai\model.py:55: UserWarning: Authentication with provided API key failed. Please double-check you provided the correct credentials.
  warnings.warn(


KeyError: 'data'